In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
!pip install addict
!pip install git+https://github.com/qubvel/segmentation_models.pytorch

In [ ]:
from addict import Dict
from pathlib import Path

data_dir = Path("/content/drive/MyDrive/Colab Notebooks/Tesi/deforestation_mapping-tesi/datadrive/forest")
process_dir = data_dir / "processed"
log_dir = data_dir / "demo/logs"

args = Dict({
    "batch_size": 8,
    "run_name": "demo", 
    "epochs": 20,
    "save_every": 50,
    "loss_type": "dice",
    "device": "cuda:0"
})

In [ ]:
%cd drive/MyDrive/Colab Notebooks/Tesi/deforestation_mapping-tesi

In [ ]:
#!/usr/bin/env python
"""
Frame to Combine Model with Optimizer

This wraps the model and optimizer objects needed in training, so that each
training step can be concisely called with a single method (optimize).
"""
from pathlib import Path
import os
import torch
import numpy as np
from torch.optim.lr_scheduler import ReduceLROnPlateau
from forest_mapping.models.metrics import *
from forest_mapping.models.reg import *
from segmentation_models_pytorch.decoders.fpn.model import *
from segmentation_models_pytorch.decoders.unet.model import *
from segmentation_models_pytorch.decoders.manet.model import *


class Framework2:
    """
    Class to Wrap all the Training Steps

    """

    def __init__(self, loss_fn=None, model_opts=None, optimizer_opts=None,
                 reg_opts=None, device=None):
        """
        Set Class Attrributes
        """
        if device is None:
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        else:
            self.device = device
        self.multi_class = True if model_opts.args.classes > 1 else False
        self.num_classes = model_opts.args.classes
        if loss_fn is None:
            if self.multi_class:
                loss_fn = torch.nn.CrossEntropyLoss()
            else:
                loss_fn = torch.nn.BCEWithLogitsLoss()
        self.loss_fn = loss_fn.to(self.device)
        if model_opts.name in ["FPN","MAnet","Unet"]:
            model_def = globals()[model_opts.name]
        else:
            raise ValueError("Unknown model name")
        print(model_def)

        self.model = model_def(**model_opts.args).to(self.device)
        optimizer_def = getattr(torch.optim, optimizer_opts.name)
        self.optimizer = optimizer_def(self.model.parameters(), **optimizer_opts.args)
        self.lrscheduler = ReduceLROnPlateau(self.optimizer, "min",
                                             verbose=True, patience=10,
                                             min_lr=1e-6)
        self.reg_opts = reg_opts


    def optimize(self, x, y):
        """
        Take a single gradient step

        Args:
            X: raw training data
            y: labels
        Return:
            optimization
        """
        x = x.permute(0, 3, 1, 2).to(self.device)
        y = y.permute(0, 3, 1, 2).to(self.device)

        self.optimizer.zero_grad()
        y_hat = self.model(x)
        loss = self.calc_loss(y_hat, y)
        loss.backward()
        self.optimizer.step()
        return y_hat.permute(0, 2, 3, 1), loss.item()

    def val_operations(self, val_loss):
        """
        Update the LR Scheduler
        """
        self.lrscheduler.step(val_loss)

    def save(self, out_dir, epoch):
        """
        Save a model checkpoint
        """
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)

        model_path = Path(out_dir, f"model_{epoch}.pt")
        optim_path = Path(out_dir, f"optim_{epoch}.pt")
        torch.save(self.model.state_dict(), model_path)
        torch.save(self.optimizer.state_dict(), optim_path)

    def infer(self, x):
        """ Make a prediction for a given x

        Args:
            x: input x

        Return:
            Prediction

        """
        x = x.permute(0, 3, 1, 2).to(self.device)
        with torch.no_grad():
            return self.model(x).permute(0, 2, 3, 1)

    def segment(self, y_hat):
        """Predict a class given logits

        Args:
            y_hat: logits output

        Return:
            Probability of class in case of binary classification
            or one-hot tensor in case of multi class"""
        if self.multi_class:
            y_hat = torch.argmax(y_hat, axis=3)
            y_hat = torch.nn.functional.one_hot(y_hat, num_classes=self.num_classes)
        else:
            y_hat = torch.sigmoid(y_hat)
        return y_hat

    def act(self, logits):
        """Applies activation function based on the model
        Args:
            y_hat: logits output
        Returns:
            logits after applying activation function"""

        if self.multi_class:
            y_hat = torch.nn.Softmax(3)(logits)
        else:
            y_hat = torch.sigmoid(logits)
        return y_hat

    def calc_loss(self, y_hat, y):
        """ Compute loss given a prediction

        Args:
            y_hat: Prediction
            y: Label

        Return:
            Loss values

        """
        y_hat = y_hat.to(self.device)
        y = y.to(self.device)

        if self.multi_class:
            y = torch.argmax(y, dim=1)
            y = torch.tensor(y, dtype=torch.long, device=self.device)

        loss = self.loss_fn(y_hat, y)

        for reg_type in self.reg_opts.keys():
            reg_fun = globals()[reg_type]
            penalty = reg_fun(
                self.model.parameters(),
                self.reg_opts[reg_type],
                self.device
            )
            loss += penalty

        return loss


    def metrics(self, y_hat, y, metrics_opts):
        """ Loop over metrics in train.yaml

        Args:
            y_hat: Predictions
            y: Labels
            metrics_opts: Metrics specified in the train.yaml

        Return:
            results

        """
        y_hat = y_hat.to(self.device)
        y = y.to(self.device)

        results = {}
        for k, metric in metrics_opts.items():
            if "threshold" in metric.keys():
                y_hat = y_hat > metric["threshold"]

            metric_fun = globals()[k]
            results[k] = metric_fun(y_hat, y)
        return results

In [ ]:
!pip install -U imagecodecs

In [ ]:
from forest_mapping.data.data import fetch_loaders
import forest_mapping.train as tr
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
from forest_mapping.models.metrics import diceloss
import yaml
import torch
import json
import segmentation_models_pytorch as smp

import ee
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import tifffile

In [ ]:
import tifffile
img=tifffile.imread('/content/drive/MyDrive/Colab Notebooks/Tesi/deforestation_mapping-tesi/datadrive/forest/processed/Training/images/S2A_MSIL2A_20200111T142701_N0213_R053_T20NQG_20200111T164651_02_11.tif')
plt.figure(figsize=(12,12))

f, bxarr = plt.subplots(2,2)
bxarr[0,0].imshow(img[...,0])
bxarr[0,1].imshow(img[...,1])
bxarr[1,0].imshow(img[...,2])
bxarr[1,1].imshow(img[...,3])

In [ ]:
!pip install earthpy

In [ ]:
# Per vedere immagine RGB del dataset sulle foreste
import earthpy.plot as ep
image = np.dstack((img[...,0],img[...,1],img[...,2]))

image2 = image.transpose(2,0,1)

ep.plot_rgb(image2,
            title="RGB Composite Image",
            stretch=True,
            str_clip=1)
plt.show()

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset as BaseDataset

In [ ]:
class Dataset(BaseDataset):
    """Amazon Dataset.

    Args:
        images_dir (str): path to images folder
        masks_dir (str): path to segmentation masks folder
        class_values (list): values of classes to extract from segmentation mask
    """
    
    CLASSES = ["No Forest", "Forest"]
    
    def __init__(
            self, 
            images_dir, 
            masks_dir, 
            classes=None,
    ):
        self.ids = os.listdir(images_dir)
        self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.ids]
        self.masks_fps = [os.path.join(masks_dir, image_id) for image_id in self.ids]
        
        # convert str names to class values on masks
        self.class_values = [float(self.CLASSES.index(cls)) for cls in classes]
        
    
    def __getitem__(self, i):

        # read data
        image = tifffile.imread(self.images_fps[i]).astype(np.float32, order='C')/ 32768.0


        # Calcolo ndvi
        red = image[...,0]
        nir = image[...,3]
        ndvi = (nir.astype(float) - red.astype(float)) / (nir + red)

        # Calcolo msavi
        msavi = nir.astype(float) + 0.5 - (0.5 * np.sqrt((2 * nir.astype(float) + 1)**2 - 8 * (nir.astype(float) - (2 * red.astype(float)))))

        image = np.dstack((image, ndvi.astype(np.float32, order='C'), msavi.astype(np.float32, order='C')))


        # extract mask
        mask_forest = tifffile.imread(self.masks_fps[i])

        mask_background = mask_forest.copy()
        mask_background[mask_background == 1] = 2
        mask_background[mask_background == 0] = 1
        mask_background[mask_background == 2] = 0
        mask = np.dstack((mask_background,mask_forest)).astype('float')
        return image, mask
        
    def __len__(self):
        return len(self.ids)

In [ ]:
conf = Dict(yaml.safe_load(open("conf/train_fpn.yaml", "r")))
train_folder = "Training"
x_train_dir = process_dir/train_folder/"images"
y_train_dir = process_dir/train_folder/"masks"
dev_folder = "Validation"
x_valid_dir = process_dir/dev_folder/"images"
y_valid_dir = process_dir/dev_folder/"masks"

In [ ]:
CLASSES = ["No Forest", "Forest"]

train_dataset = Dataset(
    x_train_dir,
    y_train_dir,
    classes = CLASSES,
)

valid_dataset = Dataset(
    x_valid_dir,
    y_valid_dir,
    classes = CLASSES,
)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False, num_workers=2)

In [ ]:
from segmentation_models_pytorch import utils

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
device = torch.device(args.device)

loss_fn = None
outchannels = conf.model_opts.args.classes
if args.loss_type == "dice":
    loss_fn = diceloss(
        act=torch.nn.Softmax(dim=1), 
        w=[0.65, 0.75],
        outchannels=outchannels, 
        label_smoothing=0.2
    )
    
frame = Framework2(
    model_opts=conf.model_opts,
    optimizer_opts=conf.optim_opts,
    reg_opts=conf.reg_opts,
    device=device,
    loss_fn=loss_fn
)

enc_name = conf.model_opts.args.encoder_name

In [ ]:
!pip install torchviz

In [ ]:
# Setup logging
writer = SummaryWriter(f"{data_dir}/{args.run_name}/fpn_vi_final_union/{enc_name}/logs/")
writer.add_text("Arguments", json.dumps(vars(args)))
writer.add_text("Configuration Parameters", json.dumps(conf))
out_dir = f"{data_dir}/{args.run_name}/models/fpn_vi_final_union/{enc_name}/"

best_epoch, best_iou = None, 0
for epoch in range(args.epochs):
    loss_d = {}
    loss_d["train"], metrics_train = tr.train_epoch(train_loader, frame, conf.metrics_opts)
    tr.log_metrics(writer, metrics_train, loss_d["train"], epoch, "train", mask_names=conf.log_opts.mask_names)
    loss_d["val"], metrics_val = tr.validate(valid_loader, frame, conf.metrics_opts)
    tr.log_metrics(writer, metrics_val, loss_d["val"], epoch, "val", mask_names=conf.log_opts.mask_names)

    # save model
    writer.add_scalars("Loss", loss_d, epoch)
    if (epoch + 1) % args.save_every == 0:
        frame.save(out_dir, epoch)
        tr.log_images(writer, frame, next(iter(train_loader)), epoch)
        tr.log_images(writer, frame, next(iter(valid_loader)), epoch, "val")

    if best_iou <= metrics_train['IoU'][0]:
        best_iou  = metrics_train['IoU'][0]
        best_epoch = epoch
        frame.save(out_dir, "best")

    print(f"{epoch}/{args.epochs} | train loss: {loss_d['train']} | val loss: {loss_d['val']} | val_recall: {metrics_val['recall'][0]}")

frame.save(out_dir, "final")
writer.close()

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
from forest_mapping.data.data import fetch_loaders
import forest_mapping.train as tr
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
from forest_mapping.models.metrics import diceloss
import yaml
import torch
import json
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import glob

In [ ]:
conf = Dict(yaml.safe_load(open("conf/train_fpn.yaml", "r")))
device = torch.device(args.device)

model_dir = f"{data_dir}/demo/models/fpn_vi_final/densenet161/model_best.pt"
outchannels = conf.model_opts.args.classes

if outchannels > 1:
    loss_weight = [1 for _ in range(outchannels)]
    loss_weight[-1] = 0 # background
    loss_fn = diceloss(act=torch.nn.Softmax(dim=1), w=loss_weight,
                               outchannels=outchannels)
else:
    loss_fn = diceloss()

frame = Framework2(
    model_opts=conf.model_opts,
    optimizer_opts=conf.optim_opts,
    reg_opts=conf.reg_opts,
    loss_fn=loss_fn,
    device=device
)
    
net = frame.model
net.load_state_dict(torch.load(model_dir))#, map_location=torch.device('cpu')
net = net.to(device)

slices_dir = f"{process_dir}/Test/images/S2*"
pred_dir = f"{process_dir}/predictions_png/fpn_vi_final/"
pred_dir2 = f"{process_dir}/predictions_numpy/fpn_vi_final/"
    
if not os.path.exists(pred_dir):
    os.makedirs(pred_dir)
if not os.path.exists(pred_dir2):
    os.makedirs(pred_dir2)

slices = glob.glob(slices_dir)

In [ ]:
total_inference_time = 0
for s in slices:
    filename = s.split("/")[-1].replace("tif","png")
    filename2 = s.split("/")[-1].replace("tif", "npy")
    

    img = tifffile.imread(s).astype(np.float32)
    # Calcolo ndvi
    red = img[...,0]      
    nir = img[...,3]
    ndvi = (nir.astype(float) - red.astype(float)) / (nir + red)

    # Calcolo msavi
    msavi = nir.astype(float) + 0.5 - (0.5 * np.sqrt((2 * nir.astype(float) + 1)**2 - 8 * (nir.astype(float) - (2 * red.astype(float)))))

    image = np.dstack((img, ndvi.astype(np.float32, order='C'), msavi.astype(np.float32, order='C')))

    
    inp_tensor=torch.from_numpy(np.expand_dims(image,axis=0)).permute((0,3,1,2)).to(device)

    start = time.time()
    output = net(inp_tensor)
    output_np = output.detach().cpu().numpy()
    output_np = np.transpose(output_np[0], (1,2,0))
    output_np = np.argmax(output_np, axis=2)
    total_inference_time += (time.time() - start)
    plt.imsave(f"{pred_dir}{filename}", output_np, vmin=0, vmax=3)
    np.save(f"{pred_dir2}{filename2}", output_np)
print(f"Total inference time: {total_inference_time}")

In [ ]:
!pip install rasterio

In [ ]:
import binascii as ba
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir datadrive/forest/demo/unet/densenet161/logs

In [ ]:
import numpy as np
from sklearn.metrics import fbeta_score, accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support, f1_score, jaccard_score, multilabel_confusion_matrix, ConfusionMatrixDisplay
import rasterio
import os

In [ ]:
#load images and use the metrics
y_true = list()
y_pred_tmp = list()
y_pred_tmp1 = list()
y_pred_tmp2 = list()

pred_png = list()


test_path = "/content/drive/MyDrive/Colab Notebooks/Tesi/deforestation_mapping-tesi/datadrive/forest/processed"
true = 'Test/masks'
pred = 'predictions_numpy/manet'

names_images = sorted(os.listdir(os.path.join(test_path,true)))
pred_png = sorted(os.listdir(os.path.join(test_path,pred)))

for image in names_images:
    path = os.path.join(test_path,true,image)

    mask_forest = tifffile.imread(path)
    mask_background = mask_forest.copy()   
    mask_background[mask_background == 1] = 2
    mask_background[mask_background == 0] = 1
    mask_background[mask_background == 2] = 0
    mask = np.dstack((mask_background,mask_forest)).astype('float')

    y_true.append(mask)

for prediction in pred_png:

    path = os.path.join(test_path,pred,prediction)
    image = np.load(path)
    y_pred_tmp.append(image)

y_true = np.asarray(y_true)
y_pred = np.asarray(y_pred_tmp)

y_true_tmp = np.zeros_like(y_pred)
y_true_tmp[y_true[:,:,:,0]==1] = 0
y_true_tmp[y_true[:,:,:,1]==1] = 1

y_true = y_true_tmp
print(y_true.shape)
print(y_pred.shape)


y_true = y_true.ravel()
y_pred = y_pred.ravel()

f1_score_teste = f1_score(y_true, y_pred,average='macro')
print("F1-score: ",f1_score_teste)

accuracy_score_teste = accuracy_score(y_true, y_pred)
print("Accuracy: ",accuracy_score_teste)

all_metrics = precision_recall_fscore_support(y_true, y_pred,average="macro")
print("Precision, recall and fscore: ", all_metrics)

iou_score_test = jaccard_score(y_true, y_pred, average=None)
print("IoU: ",iou_score_test)

cf = confusion_matrix(y_true, y_pred)
print("Confusion matrix: ")
print(cf)


fbeta = fbeta_score(y_true, y_pred, average='macro', beta=0.01)
print("Fbeta: ", fbeta)

print(classification_report(y_true, y_pred))



In [ ]:
#The following matrix shows classification percentages

mat = list()
mat.append(cf[0]/2611101)
mat.append(cf[1]/2631779)
mat = np.array(mat)
disp = ConfusionMatrixDisplay(confusion_matrix=mat, display_labels=['No Forest', 'Forest'])
disp.plot()